# TRANSFORMER - PRETRAINED MODE

-----

In [2]:
!pip install -q transformers datasets sentencepiece evaluate accelerate --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 895.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.1/362.1 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install --upgrade datasets

In [ ]:
import sys, pprint, transformers
pprint.pprint([p for p in sys.path if "transformers" in p])
print(transformers.__file__)

[]
/usr/local/lib/python3.11/dist-packages/transformers/__init__.py


In [4]:
import os, re, torch, math, random, json
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    Trainer
)

In [5]:
#Dùng mô hình của facebook, bart-base
MODEL_NAME = "facebook/bart-base"

# byte-level BPE
tok= AutoTokenizer.from_pretrained(MODEL_NAME)
model=AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME).to("cuda")

# bảo đảm có <pad>
tok.add_special_tokens({"pad_token":"<pad>"})
model.resize_token_embeddings(len(tok))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

BartScaledWordEmbedding(50265, 768, padding_idx=1)

# Tiền Xử Lý

In [6]:
raw=load_dataset("RafaelMPereira/HealthCareMagic-100k-Chat-Format-en")["train"]

def split_pairs(ds, max_samples=None):
    q, a = [], []
    for ex in ds:
        t = ex["text"]
        if "<human>:" in t and "<bot>:" in t:
            _,_,hum = t.partition("<human>:")
            hum,_,bot = hum.partition("<bot>:")
            q.append(hum.strip())
            a.append(re.sub(r"(?i)LƯU Ý.*$", "", bot).strip())
            if max_samples and len(q) >= max_samples: break
    return q, a


#max_samples
ins, outs=split_pairs(raw,max_samples=None)
# 90% train, 10% test, 10% val
tr_i,te_i,tr_o,te_o=train_test_split(ins, outs,test_size=0.1,random_state=42)
tr_i,va_i,tr_o,va_o=train_test_split(tr_i, tr_o,test_size=0.1,random_state=42)

README.md:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

HealthCareMagic-100k-en.jsonl:   0%|          | 0.00/125M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/112165 [00:00<?, ? examples/s]

In [7]:
print(len(ins))

112165


# Lớp Dataset riêng cho pretrained trans

In [8]:
MAX_LEN = 256
class BartQADataset(torch.utils.data.Dataset):
    def __init__(self, q, a, tok):
        self.q, self.a, self.tok = q, a, tok
    def __len__(self): return len(self.q)
    def __getitem__(self, i):
        src = f"Patient: {self.q[i]}"
        tgt = f"{self.a[i]}"
        enc = self.tok(src, truncation=True, max_length=MAX_LEN)
        dec = self.tok(tgt, truncation=True, max_length=MAX_LEN)
        return {"input_ids": enc["input_ids"],
                "attention_mask": enc["attention_mask"],
                "labels": dec["input_ids"]}
train_ds = BartQADataset(tr_i, tr_o, tok)
val_ds   = BartQADataset(va_i, va_o, tok)
test_ds  = BartQADataset(te_i, te_o, tok)

collator = DataCollatorForSeq2Seq(tok, model=model)

In [7]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    output_dir="ft-bart-bpe",
    #Dùng 2 epoch <-- Mô hình qúa mạnh rồi, chạy lâu nữa nên ít thôi
    num_train_epochs=2,

    #Dùng 4 batch size cho mỗi GPU <- nhẹ
    per_device_train_batch_size=4,

    #Tích lũy gradient trước khi update
    gradient_accumulation_steps=4,
    #Nhìn chung kích thước là 4 x 4 = 16

    learning_rate=3e-5,
    warmup_steps=500,

    #Đánh giá trên validation mỗi epoch
    eval_strategy="epoch",

    #Lưu checkpoint mỗi epoch
    save_strategy="epoch",

    #In ra loss/metric sau mỗi 50step/1 epoch
    logging_steps=50,
    save_total_limit=2,

    # load model tốt nhất cuối nhất
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    greater_is_better=False,

    #Mixed-precision-GPU hỗ trợ
    fp16=True,
    report_to="none",
)


trainer = Trainer(
    model=model, args=args,
    train_dataset=train_ds, eval_dataset=val_ds,
    data_collator=collator
)
trainer.train()
model.save_pretrained("ft-bart-bpe"); tok.save_pretrained("ft-bart-bpe")

Epoch,Training Loss,Validation Loss
1,2.750100,2.548423
2,2.659500,2.469687


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3463: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


('ft-bart-bpe/tokenizer_config.json',
 'ft-bart-bpe/special_tokens_map.json',
 'ft-bart-bpe/vocab.json',
 'ft-bart-bpe/merges.txt',
 'ft-bart-bpe/added_tokens.json',
 'ft-bart-bpe/tokenizer.json')

# Đánh giá trên tập Test

In [ ]:
@torch.no_grad()
def bart_reply(question, max_new=128, top_p=0.9, temp=0.7):
    prompt = f"Patient: {question}"
    inps = tok(prompt, return_tensors="pt").to(model.device)
    out = model.generate(**inps, max_new_tokens=max_new,
                         do_sample=True, top_p=top_p,
                         temperature=temp, eos_token_id=tok.eos_token_id)
    ans = tok.decode(out[0], skip_special_tokens=True)
    # cắt phần prompt ra
    return ans.replace(prompt, "").lstrip()

In [9]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=3f8a71cddbbe00178bebce0e7c558c87ca21ef5e4d014de16d648d36bc54b0d5
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [25]:
from transformers import AutoModelForCausalLM

#Load tokenizer
tok = AutoTokenizer.from_pretrained("ft-bart-bpe/checkpoint-11218", use_fast=True)

#đảm bảo có pad token
if tok.pad_token is None:
    tok.add_special_tokens({"pad_token":"<pad>"})

#load lại model
base = AutoModelForCausalLM.from_pretrained("gpt2")
base = base.to(device)

model = AutoModelForCausalLM.from_pretrained("ft-bart-bpe/checkpoint-11218").to(device)
model.eval()

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Some weights of BartForCausalLM were not initialized from the model checkpoint at ft-bart-bpe/checkpoint-11218 and are newly initialized: ['lm_head.weight', 'model.decoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BartForCausalLM(
  (model): BartDecoderWrapper(
    (decoder): BartDecoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartDecoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (activation_fn): GELUActivation()
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (encoder_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_feat

In [ ]:
bleu  = evaluate.load("bleu")
rouge = evaluate.load("rouge")

In [ ]:
# Chuẩn bị test set (theo split bạn đã chia)
hyps, refs = [], []

for prompt, gold in zip(te_i, te_o):
    #Input
    text_in = f"Patient: {prompt}\nDoctor:"
    enc     = tok(text_in, return_tensors="pt", truncation=True,
                  padding=True, max_length=256).to(device)

    #Sinh trả lời
    out_ids = model.generate(
        enc.input_ids,
        attention_mask=enc.attention_mask,
        max_new_tokens=128,
        #beam search
        num_beams=4,
        early_stopping=True
    )
    pred = tok.decode(out_ids[0], skip_special_tokens=True)
    #cắt phần prompt để chỉ lấy portion sau "Doctor:"
    summary = pred.split("Doctor:")[-1].strip()

    hyps.append(summary)
    refs.append([gold])

In [28]:
#Tính BLEU và ROUGE‐L
bleu_score=bleu.compute(predictions=hyps,references=refs)["bleu"]*100
rouge_score=rouge.compute(predictions=hyps, references=[r[0] for r in refs])["rougeL"]*100

print(f"BLEU-4:   {bleu_score:.2f}")
print(f"ROUGE-L:  {rouge_score:.2f}")

BLEU-4:  0.03
ROUGE-L: 0.23


In [29]:
from google.colab import drive
import shutil, os

#Mount Google Drive
drive.mount('/content/drive', force_remount=True)
src_folder = '/content/ft-bart-bpe/checkpoint-11218'
dst_folder = '/content/drive/MyDrive/LNP_OUT/checkpoint-11218_transformer_pretrained'
os.makedirs(os.path.dirname(dst_folder), exist_ok=True)
#COPy lại thư mục
shutil.copytree(src_folder, dst_folder)

Mounted at /content/drive


'/content/drive/MyDrive/LNP_OUT/checkpoint-11218_transformer_pretrained'

# SỬA

In [12]:
import math
import evaluate
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# try:
#     from google.colab import drive
#     drive.mount('/content/drive')
# except ImportError:
#     print("Không phải Colab, bỏ qua bước mount. Hãy mount Drive theo cách của bạn nếu cần.")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dst_folder = '/content/drive/MyDrive/LNP_OUT/checkpoint-11218_transformer_pretrained'

tok = AutoTokenizer.from_pretrained(dst_folder, use_fast=True)
if tok.pad_token is None:
    tok.add_special_tokens({"pad_token": "<pad>"})
model = AutoModelForSeq2SeqLM.from_pretrained(dst_folder).to(device)
model.config.decoder_start_token_id = tok.bos_token_id
model.eval()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_lay

In [13]:
bleu  = evaluate.load("bleu")
rouge = evaluate.load("rouge")

#Tính Log-PPL và PPL
nll, n_tokens = 0.0, 0
for prompt, gold in zip(te_i, te_o):
    inputs = tok(f"Patient: {prompt}",
                 truncation=True, max_length=256,
                 return_tensors="pt").to(device)
    with tok.as_target_tokenizer():
        labels = tok(gold,
                     truncation=True, max_length=256,
                     return_tensors="pt").input_ids.to(device)
    with torch.no_grad():
        outputs = model(input_ids=inputs.input_ids,
                        attention_mask=inputs.attention_mask,
                        labels=labels)
    non_pad = (labels != tok.pad_token_id).sum().item()
    nll    += outputs.loss.item() * non_pad
    n_tokens += non_pad

log_ppl = nll / n_tokens
ppl     = math.exp(log_ppl)

#tính BLEU, ROUGE
hyps, refs = [], []
for prompt, gold in zip(te_i, te_o):
    inputs = tok(f"Patient: {prompt}",
                 return_tensors="pt", truncation=True, max_length=256
                ).to(device)

    # Greedy decoding (mặc định)
    # Beam search (num_beams>1)
    # Sampling (do_sample=True, kết hợp top_k, top_p, temperature, v.v.)
    out_ids = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_length=256,
        num_beams=4,
        early_stopping=True,
        pad_token_id=tok.pad_token_id,
        eos_token_id=tok.eos_token_id
    )
    pred    = tok.decode(out_ids[0], skip_special_tokens=True)
    summary = pred.split("Doctor:")[-1].strip()
    hyps.append(summary)
    refs.append([gold])

bleu_score  = bleu.compute(predictions=hyps, references=refs)["bleu"] * 100
rouge_score = rouge.compute(predictions=hyps, references=[r[0] for r in refs])["rougeL"] * 100

print(f"Log-PPL: {log_ppl:.4f} | PPL: {ppl:.2f}")
print(f"BLEU-4 : {bleu_score:.2f}")
print(f"ROUGE-L: {rouge_score:.2f}")

Log-PPL: 2.5079 | PPL: 12.28
BLEU-4 : 8.39
ROUGE-L: 18.75


In [ ]:
import random
import datetime
import os
import textwrap

In [14]:
ARTIFACT_DIR = "artifacts"
os.makedirs(ARTIFACT_DIR, exist_ok=True)

# 25 mẫu test ngẫu nhiên
rand_idx = random.sample(range(len(te_i)), 25)
out_path = os.path.join(ARTIFACT_DIR, f"human_eval_bart_{datetime.date.today()}.txt")

with open(out_path, "w", encoding="utf-8") as fp:
    for k, idx in enumerate(rand_idx, 1):
        q, ref = te_i[idx], te_o[idx]
        # Sinh câu trả lời
        inputs = tok(f"Patient: {q}", return_tensors="pt",
                     truncation=True, max_length=256).to(device)
        out_ids = model.generate(
            inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_length=256,
            num_beams=4,
            early_stopping=True,
            pad_token_id=tok.pad_token_id,
            eos_token_id=tok.eos_token_id,
        )
        pred = tok.decode(out_ids[0], skip_special_tokens=True)
        pred = pred.split("Doctor:")[-1].strip()

        fp.write("=" * 80 + "\n")
        fp.write(f"[{k:02d}] QUESTION:\n{textwrap.fill(q, 70)}\n\n")
        fp.write(f"REFERENCE:\n{textwrap.fill(ref, 70)}\n\n")
        fp.write(f"MODEL:\n{textwrap.fill(pred, 70)}\n\n")

print(f"Đã xuất 25 mẫu human‐eval BART lên {out_path}")

Đã xuất 25 mẫu human‐eval BART lên artifacts/human_eval_bart_2025-05-21.txt
